In [ ]:
import warnings
import os

# Ustawiono środowiskowe zmienne konfiguracyjne TensorFlow, aby wyłączyć OneDNN i ograniczyć logi
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

# Załadowano niezbędne biblioteki do przetwarzania danych, budowy modelu i wizualizacji wyników
import zipfile
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
import tarfile
import urllib.request
import shutil

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.applications import VGG16
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

# Zdefiniowano niestandardową warstwę BLIP do generowania opisów i streszczeń na podstawie obrazów
class BlipCaptionSummaryLayer(tf.keras.layers.Layer):
    def __init__(self, processor, model, **kwargs):
        """
        Inicjalizacja niestandardowej warstwy Keras z wykorzystaniem BLIP processor oraz modelu.
        """
        super().__init__(**kwargs)
        self.processor = processor
        self.model = model

    def call(self, image_path, task):
        # Wykorzystano tf.py_function do wykonania niestandardowego przetwarzania obrazu oraz generacji tekstu
        return tf.py_function(self.process_image, [image_path, task], tf.string)

    def process_image(self, image_path, task):
        """
        Wczytano obraz, wykonano jego przetwarzanie oraz wygenerowano opis.
        """
        try:
            # Zdekodowano ścieżkę obrazu z tensora TensorFlow na string
            image_path_str = image_path.numpy().decode("utf-8")
            # Otworzono obraz z wykorzystaniem PIL i skonwertowano na format RGB
            image = Image.open(image_path_str).convert("RGB")

            # W zależności od zadania przypisano odpowiedni prompt
            if task.numpy().decode("utf-8") == "caption":
                prompt = "This is a picture of"
            else:
                prompt = "This is a detailed photo showing"

            # Przygotowano wejście do modelu BLIP przy użyciu processor'a
            inputs = self.processor(images=image, text=prompt, return_tensors="pt")

            # Wygenerowano wyjście tekstowe przy użyciu modelu BLIP
            output = self.model.generate(**inputs)
            # Zdekodowano wynik do czytelnego stringa
            result = self.processor.decode(output[0], skip_special_tokens=True)
            return result
        except Exception as e:
            # W przypadku błędu wyświetlono komunikat i zwrócono informację o błędzie
            print(f"Error: {e}")
            return "Error processing image"

def generate_text(image_path, task):
    """
    Funkcja generująca tekst (opis/streszczenie) dla obrazu.
    Tworzona jest instancja niestandardowej warstwy BLIP i wykonywane są odpowiednie przekształcenia.
    """
    blip_layer = BlipCaptionSummaryLayer(BlipProcessor, model)
    return blip_layer(image_path, task)

def plot_image_with_title(image, model, true_label, predicted_label, class_names):
    """
    Wyświetlono pojedynczy obraz wraz z prawidłową etykietą oraz predykcją modelu.
    """
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    # Przypisano nazwy klas na podstawie indeksów
    true_label_name = class_names[true_label]
    pred_label_name = class_names[predicted_label]
    plt.title(f"True: {true_label_name}\nPred: {pred_label_name}")
    plt.axis('off')
    plt.show()

def test_model_on_image(test_generator, model, index_to_plot=0):
    """
    Przeprowadzono test modelu na wybranym obrazie z zestawu testowego.
    Pobierana jest partia obrazów, wyliczane są predykcje i wyświetlany jest obraz wraz z etykietami.
    """
    test_images, test_labels = next(test_generator)
    predictions = model.predict(test_images)
    predicted_classes = (predictions > 0.5).astype(int).flatten()
    class_indices = test_generator.class_indices
    class_names = {v: k for k, v in class_indices.items()}
    image_to_plot = test_images[index_to_plot]
    true_label = test_labels[index_to_plot]
    predicted_label = predicted_classes[index_to_plot]
    plot_image_with_title(image=image_to_plot, model=model, true_label=true_label, predicted_label=predicted_label, class_names=class_names)

# Ustalono ziarno dla powtarzalności wyników
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Ustalono parametry treningu oraz rozmiar przetwarzanych obrazów
batch_size = 32
n_epochs = 5
img_rows, img_cols = 224, 224
input_shape = (img_rows, img_cols, 3)

# Pobrano zbiór danych z archiwum tar
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/ZjXM4RKxlBK9__ZjHBLl5A/aircraft-damage-dataset-v1.tar"
tar_filename = "aircraft_damage_dataset_v1.tar"
extracted_folder = "aircraft_damage_dataset_v1"

urllib.request.urlretrieve(url, tar_filename)
print(f"Pobrano {tar_filename}. Rozpoczęto ekstrakcję zawartości.")

if os.path.exists(extracted_folder):
    print(f"Folder '{extracted_folder}' już istnieje. Usunięto istniejący folder w celu uniknięcia duplikacji.")
    shutil.rmtree(extracted_folder)
    print(f"Usunięto folder: {extracted_folder}")

with tarfile.open(tar_filename, "r") as tar_ref:
    tar_ref.extractall()
    print(f"Rozpakowano {tar_filename} pomyślnie.")

# Zdefiniowano katalogi dla podziału na zbiory treningowy, walidacyjny oraz testowy
extract_path = "aircraft_damage_dataset_v1"
train_dir = os.path.join(extract_path, 'train')
test_dir = os.path.join(extract_path, 'test')
valid_dir = os.path.join(extract_path, 'valid')

# Utworzono ImageDataGenerator'y do przetwarzania danych obrazowych
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    seed=seed_value,
    class_mode='binary',
    shuffle=True
)

valid_generator = valid_datagen.flow_from_directory(
    directory=valid_dir,
    class_mode='binary',
    seed=seed_value,
    batch_size=batch_size,
    shuffle=False,
    target_size=(img_rows, img_cols)
)

test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    class_mode='binary',
    seed=seed_value,
    batch_size=batch_size,
    shuffle=False,
    target_size=(img_rows, img_cols)
)

# Wczytano bazowy model VGG16 z wagami z ImageNet bez warstwy w pełni połączonej
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))
output = base_model.layers[-1].output
output = keras.layers.Flatten()(output)
base_model = Model(base_model.input, output)

# Zamrożono warstwy modelu VGG16
for layer in base_model.layers:
    layer.trainable = False

# Zbudowano model własny, dodając kolejne warstwy gęste, Dropout oraz warstwę wyjściową
model = Sequential()
model.add(base_model)
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Przeprowadzono trening modelu przy użyciu generatora danych treningowych oraz walidacyjnych
history = model.fit(
    train_generator,
    epochs=n_epochs,
    validation_data=valid_generator
)

train_history = model.history.history

# Wykres przedstawiający stratę treningową
plt.title("Training Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.plot(train_history['loss'])
plt.show()

# Wykres przedstawiający stratę walidacyjną
plt.title("Validation Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.plot(train_history['val_loss'])
plt.show()

# Wykres dokładności treningowej i walidacyjnej
plt.plot(train_history['accuracy'], label='Training Accuracy')
plt.plot(train_history['val_accuracy'], label='Validation Accuracy')
plt.title("Accuracy Curve")
plt.ylabel("Epochs")
plt.xlabel("Accuracy")
plt.legend()
plt.show()

# Ewaluacja modelu na zbiorze testowym
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Testowanie modelu na wybranym obrazie ze zbioru testowego
test_model_on_image(test_generator, model, index_to_plot=1)

# Wczytano przetrenowany processor oraz model BLIP z repozytorium Salesforce
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Ścieżka do przykładowego obrazu z zestawu testowego
image_path = tf.constant("aircraft_damage_dataset_v1/test/dent/144_10_JPG_jpg.rf.4d008cc33e217c1606b76585469d626b.jpg")

# Wygenerowano opis dla obrazu i wyświetlono wynik
caption = generate_text(image_path, tf.constant("caption"))
print("Caption:", caption.numpy().decode("utf-8"))

# Wygenerowano streszczenie dla obrazu i wyświetlono wynik
summary = generate_text(image_path, tf.constant("summary"))
print("Summary:", summary.numpy().decode("utf-8"))

# Wczytano kolejny obraz w celu wyświetlenia opisu i streszczenia
image_url = "aircraft_damage_dataset_v1/test/dent/149_22_JPG_jpg.rf.4899cbb6f4aad9588fa3811bb886c34d.jpg"
img = plt.imread(image_url)
plt.imshow(img)
plt.axis('off')
plt.show()

image_path = tf.constant("aircraft_damage_dataset_v1/test/dent/149_22_JPG_jpg.rf.4899cbb6f4aad9588fa3811bb886c34d.jpg")
caption = generate_text(image_path, tf.constant("caption"))
print("Caption:", caption.numpy().decode("utf-8"))
summary = generate_text(image_path, tf.constant("summary"))
print("Summary:", summary.numpy().decode("utf-8"))
